In [72]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
#using matplotlib’s ggplot style
plt.style.use('ggplot')
import seaborn as sns
sns.set(color_codes=True)
from sklearn.preprocessing import MinMaxScaler
from sklearn import decomposition,linear_model
from sklearn.model_selection import cross_val_score,cross_val_predict,GridSearchCV
from sklearn.linear_model import RidgeClassifier,Lasso,LogisticRegression,LinearRegression,Ridge
from sklearn.metrics import  mean_squared_error,confusion_matrix,accuracy_score,recall_score,f1_score,roc_auc_score,precision_recall_curve,roc_curve,auc
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from sklearn.pipeline import Pipeline
from pprint import pprint
from time import time
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression



In [73]:
path ='D:/python_notebook/capstone/'

In [74]:
total = pd.read_csv(path + 'final_data(industry not added).csv')
extra_data1=pd.read_csv('./extra_feature1.csv')
extra_data2=pd.read_csv('./extra_feature2.csv')
extra_data=pd.merge(extra_data1,extra_data2,on=['year','state'],how='outer')
total=pd.merge(total,extra_data,on=['year','state'],how='left')
total=total.loc[total.GeoName!='District of Columbia',:]
total_merge=total.copy()


In [75]:
#generate target variable
new_target=pd.read_csv('./tagert.csv')
new_target=new_target.drop_duplicates()
new_target['opn_diff']=(new_target.open_establish-new_target.end_establish)
new_target['exp_diff']=(new_target.expand_establish-new_target.contract_establish)
new_target['total_diff']=(new_target.total_expand-new_target.total_contract)
new_target=new_target[['state','year','opn_diff','exp_diff','total_diff',
                       'open_establish','expand_establish','total_expand',
                       'end_establish','contract_establish','total_contract']]
multi_target=pd.DataFrame(new_target.groupby(['state','year'])[['opn_diff','exp_diff','total_diff',
                       'open_establish','expand_establish','total_expand',
                       'end_establish','contract_establish','total_contract']].sum()).reset_index()
print(multi_target.shape)
mask=(multi_target.state!='District of Columbia')&(multi_target.state!='Puerto Rico')&(multi_target.state!='Virgin Islands')
multi_target=multi_target.loc[mask,:]
print(multi_target.shape)
#target_df=target_df.reset_index(drop=True)
#target_df =target_df.set_index(['year'])


(1370, 11)
(1300, 11)


In [76]:
multi_target=multi_target.set_index(['year'])

In [77]:
features =total.drop(columns=['contract_establish','expand_establish','end_establish',
                              'net_change','open_establish','total_contract','total_expand',
                              'GeoName','state','year'])

In [78]:
print('feature number before dropping multicollinearity',features.shape)
corr_matrix =features.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# Drop features 
final_features =features.drop(columns=to_drop, axis=1)
print('feature number after dropping multicollinearity',final_features.shape)

feature number before dropping multicollinearity (1000, 213)
feature number after dropping multicollinearity (1000, 123)


In [79]:
state_list =total.GeoName.unique().tolist()
total =total.drop(columns =['contract_establish','expand_establish','end_establish',
                              'net_change','open_establish','total_contract','total_expand'])
print('dropping high correlation features')
print(total.shape)
total =total.drop(columns=to_drop, axis=1)
print(total.shape)
print('dropping process finished the final feature numbers')

dropping high correlation features
(1000, 216)
(1000, 126)
dropping process finished the final feature numbers


In [80]:
def clean_state_data(total):
    #df_1 =total_3.loc[total_3['GeoName'] == st_name]
    df =total.copy()
    df =df.set_index(['GeoName','year'])
    df =df.replace({'(NA)': np.nan ,'(L)':np.nan ,0:np.nan})
    df =df.drop(columns =['state'])
    #check missing value 
    miss_col = df
    miss_col = miss_col.isnull().sum(axis =0)/float(len(miss_col))
    miss_col =miss_col.sort_values(ascending=False)
    columns = df.columns
    missing_value_df = pd.DataFrame({'percent_missing': miss_col})
    missing_value_df['percent_missing'] =missing_value_df['percent_missing'].astype('float64')
    
    miss_col =missing_value_df.loc[(missing_value_df['percent_missing'] >0) 
                               &(missing_value_df['percent_missing'] <0.3)].index.tolist()  
    #drop columns with large missing
    del_list = missing_value_df.loc[(missing_value_df['percent_missing'] >=0.3)].index.tolist()
    df = df.drop(columns=del_list)
    df=df.astype(float)
    df=df.apply(lambda x: x.fillna(x.mean()),axis=0)
    df_2 = df.copy()
    #numerical feature scaling
    scaler = MinMaxScaler()
    col =df_2.columns.values
    df_2[col] = scaler.fit_transform(df_2[col]) 
    return df_2

In [81]:
total =clean_state_data(total)

In [82]:
state_group=total.groupby('GeoName')
group_count=0
for name, group in state_group:
    shift1=group.shift(-1)
    shift2=group.shift(-2)
    change_rate=shift1-group
    velocity_rate=shift2-shift1
    change_rate=change_rate.reset_index()
    velocity_rate=velocity_rate.reset_index()
    change_rate['year']=list(range(1998,2018,1))
    velocity_rate['year']=list(range(1999,2019,1))
    change_col_name=[x if x=='GeoName' or x=='year' else 'change_rate:'+x for x in change_rate.columns ]
    change_rate.columns=change_col_name
    velocity_col_name=[x if x=='GeoName' or x=='year' else 'velocity of change:'+x for x in velocity_rate.columns ]
    velocity_rate.columns=velocity_col_name
    if group_count==0:
        change_rate_df=change_rate
        velocity_rate_df=velocity_rate
    else:
        change_rate_df=change_rate_df.append(change_rate)
        velocity_rate_df=velocity_rate_df.append(velocity_rate)
    group_count+=1  
total_1year=pd.merge(total,change_rate_df,on=['GeoName','year'],how='inner')
total_2year=pd.merge(total_1year,velocity_rate_df,on=['GeoName','year'])

In [83]:
def generate_X(total,lag_year):
    total=total.set_index(['GeoName','year'])
    total=total.reset_index(level='GeoName')
    x_train_1 =total.loc[list(range(1997+lag_year,2012))]
    x_test_1 =total.loc[[2012,2013,2014,2015]]
    x_train =x_train_1.drop(columns =['GeoName'])
    x_test  =x_test_1.drop(columns =['GeoName'])
    x_predict=total.loc[[2016]]
    print(x_train.shape)
    print(x_test.shape)
    return x_train,x_test

def generate_Y(target_df,col_name,lag_year):
    y_train=target_df.loc[list(range(1997+lag_year,2012)),col_name]
    y_test =target_df.loc[[2013,2014,2015,2016],col_name]
    return y_train,y_test


In [84]:
x_train_1,x_test_1=generate_X(total_1year,1)
x_train_2,x_test_2=generate_X(total_2year,2)

(700, 230)
(200, 230)
(650, 345)
(200, 345)


In [85]:
y_list=['opn_diff','exp_diff','total_diff',
                       'open_establish','expand_establish','total_expand',
                       'end_establish','contract_establish','total_contract']
y_dict={}
for y in y_list:
    temp_data=[]
    for lag_year in [1,2]:
        temp_y_train,temp_y_test=generate_Y(multi_target,y,lag_year)
        temp_data.append(temp_y_train)
        temp_data.append(temp_y_test)
        print(temp_y_train.shape)
        print(temp_y_test.shape)
    y_dict[y]=temp_data
y_dict.keys()

(700,)
(200,)
(650,)
(200,)
(700,)
(200,)
(650,)
(200,)
(700,)
(200,)
(650,)
(200,)
(700,)
(200,)
(650,)
(200,)
(700,)
(200,)
(650,)
(200,)
(700,)
(200,)
(650,)
(200,)
(700,)
(200,)
(650,)
(200,)
(700,)
(200,)
(650,)
(200,)
(700,)
(200,)
(650,)
(200,)


dict_keys(['opn_diff', 'exp_diff', 'total_diff', 'open_establish', 'expand_establish', 'total_expand', 'end_establish', 'contract_establish', 'total_contract'])

In [86]:
Lr=LinearRegression()
lasso=Lasso()
ridge=Ridge()
rfc=RandomForestRegressor()
xgb=XGBRegressor()
Kbest=SelectKBest(f_regression, k=20)
lr_pipe=Pipeline([('Kbest',Kbest),('lr',Lr)])
lr_parameters={
                'Kbest__k':[20,30,40]
}
lasso_parameters={
                'alpha':[0.01,0.1,1,10]}
rfc_parameters= {
                 'max_depth': [7],
                 #'max_depth': [5,7,11,15],
                 #'max_features':[0.02,0.05,0.1,0.15], 
                'max_features':[0.05,0.1], 
                'n_estimators':[10]
                #'n_estimators':[10,15,20]
             }
ridge_parameters={
                'alpha':[0.01,0.1,1,10]
}

xgb_parameters = {
                #'min_child_weight':range(1,6,2),
                'min_child_weight':[5],
                #'max_depth': [5,10,15],
                'max_depth': [10],
                'subsample':[0.7],
                #'subsample':[0.6,0.7],
                'objective':['reg:linear'],
                'learning_rate':[0.1],
                'n_estimators':[150],
                'gamma':[0.4]
             }
model_dict={'lr':[lr_pipe,lr_parameters],'lasso':[lasso,lasso_parameters],
            'ridge':[ridge,ridge_parameters],'rfc':[rfc,rfc_parameters],'xgb':[xgb,xgb_parameters]}
                                 
model_dict1={'rfc':[rfc,rfc_parameters],'xgb':[xgb,xgb_parameters]}

In [87]:
def parameter_tunning(model,parameter_dict,x_train,y_train):
    grid_clf = GridSearchCV(model, parameter_dict, cv=5,scoring='neg_mean_squared_error',n_jobs=4)
    gcv=grid_clf.fit(x_train, y_train)
    best_parameters=gcv.best_params_
    best_scores=gcv.best_score_
    #print(best_scores)
    return best_parameters

In [88]:
def perfomance_model(x_train,y_train,x_test,y_test,regressor):
    regressor.fit(x_train,y_train)
    y_test_pred=regressor.predict(x_test)
    y_train_pred=regressor.predict(x_train)
    mse=mean_squared_error(y_test_pred,y_test)
    return mse


In [94]:
for target,data in y_dict.items():
    print(target)
    performance_dct={}
    for model_name,model in model_dict.items():
        print('tunning parameters for '+str(model_name))
        param1=parameter_tunning(model[0],model[1],x_train_1,data[0])
        param2=parameter_tunning(model[0],model[1],x_train_2,data[2])
        mse_1=perfomance_model(x_train_1,data[0],x_test_1,data[1],model[0].set_params(**param1))
        mse_2=perfomance_model(x_train_2,data[2],x_test_2,data[3],model[0].set_params(**param2))
        result_list=[mse_1,mse_2]
        performance_dct[model_name]=result_list  
        performance_df=pd.DataFrame(performance_dct)
        performance_df=performance_df.T
        performance_df.columns=['mse with change features','mse with rate of change features']
    display(performance_df)

opn_diff
tunning parameters for lr
tunning parameters for lasso
tunning parameters for ridge
tunning parameters for rfc
tunning parameters for xgb


,mse with change features,mse with rate of change features
lr,2.206336e+16,9.288366e+08
lasso,1.348598e+08,1.233339e+08
ridge,1.361297e+08,1.488001e+08
rfc,2.095683e+08,1.881445e+08
xgb,1.468561e+08,1.758504e+08


exp_diff
tunning parameters for lr
tunning parameters for lasso
tunning parameters for ridge
tunning parameters for rfc
tunning parameters for xgb


,mse with change features,mse with rate of change features
lr,2.494244e+17,3.765196e+09
lasso,5.907832e+11,5.435685e+11
ridge,1.543607e+10,2.751956e+10
rfc,2.454486e+09,3.955809e+09
xgb,3.256801e+09,3.811726e+09


total_diff
tunning parameters for lr
tunning parameters for lasso
tunning parameters for ridge
tunning parameters for rfc
tunning parameters for xgb


...

,mse with change features,mse with rate of change features
lr,3.629350e+16,6.901277e+16
lasso,1.115249e+09,1.280486e+09
ridge,9.097556e+08,9.889702e+08
rfc,2.098225e+09,2.665354e+09
xgb,5.970404e+08,7.136242e+08


contract_establish
tunning parameters for lr
tunning parameters for lasso
tunning parameters for ridge
tunning parameters for rfc
tunning parameters for xgb


,mse with change features,mse with rate of change features
lr,2.217240e+16,1.024806e+17
lasso,7.451255e+10,8.141061e+10
ridge,8.606061e+09,4.828556e+09
rfc,1.250492e+10,2.734062e+10
xgb,1.462964e+10,2.078231e+10


total_contract
tunning parameters for lr
tunning parameters for lasso
tunning parameters for ridge
tunning parameters for rfc
tunning parameters for xgb


,mse with change features,mse with rate of change features
lr,7.396821e+15,1.008228e+14
lasso,5.256089e+10,6.155794e+10
ridge,9.794532e+09,5.565392e+09
rfc,4.160525e+10,3.013355e+10
xgb,2.936909e+10,2.992489e+10


In [92]:
# using linear regressiong tends to use large K to 

{'Kbest__k': 40}

{'Kbest__k': 30}